In [10]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from data_collection import DataCollection
%matplotlib inline


In [11]:
ROOT_DIR = os.path.realpath(os.path.abspath(''))
df_out = pd.read_csv(ROOT_DIR + "/../data/output/res_20190610.csv_aggregate_output.csv")

In [12]:
df_out.head()

,timestamp,price,side,bp0,bq0,bp1,bq1,bp2,bq2,bp3,...,ap0,aq0,ap1,aq1,ap2,aq2,ap3,aq3,ap4,aq4
0,0,9990,b,9990,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,9990,b,9990,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,9995,a,9990,27,0,0,0,0,0,...,9995,1,0,0,0,0,0,0,0,0
3,0,9990,b,9990,33,0,0,0,0,0,...,9995,1,0,0,0,0,0,0,0,0
4,0,9990,b,9990,34,0,0,0,0,0,...,9995,1,0,0,0,0,0,0,0,0


# Calculating our feature set

For more information on implementation, view ./data_collection.py

In [19]:
spread = df_out.apply(DataCollection.calc_bid_ask_spread, 1)
current_price = df_out.apply(DataCollection.calc_going_rate, 1)
vwap_a = df_out.apply(DataCollection.calc_vwap_a, 1)
vwap_b = df_out.apply(DataCollection.calc_vwap_b, 1)
imbalance = df_out.apply(DataCollection.calc_imbal, 1)
volume = df_out.apply(DataCollection.calc_volume, 1)


In [20]:
target = DataCollection.calc_target(current_price)

Raw feature set

In [14]:
feature_set = pd.DataFrame({
    "timestamp": df_out['timestamp'],
    "going_rate": current_price,
    "b/a_spread": spread,
    "ask_vol_weight_avg_price": vwap_a,
    "bid_vol_weight_avg_price": vwap_b,
    "order_imbalance": imbalance,
    "volume": volume,
    "target": target
})

We will remove the first two, and last rows from the feature set. This is because, the first two rows have measures for the going rate. The last row will be removed because the target is arbitrarily set to 'no_change'

In [15]:
feature_set_cleaned = feature_set[3:-1]

In [16]:
feature_set_cleaned.head()

,timestamp,going_rate,b/a_spread,ask_vol_weight_avg_price,bid_vol_weight_avg_price,order_imbalance,volume,target
3,0,9992.5,5,9995.0,9990.000000,1.0,66,None
4,0,9992.5,5,9995.0,9990.000000,1.0,68,None
5,0,9992.5,5,9995.0,9989.857143,0.5,69,None
6,0,9992.5,5,9997.5,9989.857143,1.0,70,None
7,0,9992.5,5,9997.5,9989.722222,1.0,72,None
